In [1]:
from dotenv import load_dotenv
from metagame import TradingClient
import os

load_dotenv()

client = TradingClient(
    api_url=os.environ["API_URL"],
    jwt=os.environ["JWT"],
    act_as=int(0),
)
client.act_as(2)

ActingAs(account_id=2)

In [2]:
import pandas as pd

def get_market_trades(market_name: str):
    state = client.state()
    market_id = state.market_name_to_id.get(market_name)    
    if market_id is None:
        return pd.DataFrame()

    if not state.markets[market_id].hasFullTradeHistory:
        client.get_full_trade_history(market_id)
        state = client.state()

    trades = state.markets[market_id].trades
    if not trades:
        return pd.DataFrame()
    df = pd.DataFrame(trades).set_index("id")
    df["buyer"] = df["buyer_id"].map(lambda x: state.accounts[x].name)
    df["seller"] = df["seller_id"].map(lambda x: state.accounts[x].name)
    df["timestamp"] = df["transaction_id"].map(lambda x: state.transactions[x])
    return df.drop(columns=["market_id", "buyer_id", "seller_id", "transaction_id"])


In [11]:
state = client.state()
markets = list(state.markets.values())
df = pd.DataFrame(state.portfolios.values())
df["account_name"] = df["account_id"].map(lambda x: state.accounts[x].name)
for market in markets:
    df[f"{market.definition.name}_position"] = df.market_exposures.map(lambda xs: next((x["position"] for x in xs if x["market_id"] == market.definition.id), 0))
df.drop(columns=["account_id", "market_exposures", "owner_credits"])


,total_balance,available_balance,account_name,test_bond_position,alpha_tradewars_position,bravo_tradewars_position,charlie_tradewars_position,delta_tradewars_position,echo_tradewars_position,foxtrot_tradewars_position,golf_tradewars_position,hotel_tradewars_position,india_tradewars_position,bond_tradewars_position,abc_tradewars_position,def_tradewars_position,ghi_tradewars_position
0,9.980001e+09,9.974061e+09,Nicholas Charette,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-1.18,0.00,0.0,0.00
1,0.000000e+00,0.000000e+00,tradewars_3,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
2,4.686460e+03,2.358460e+03,tradewars_1,0,0.20,0.00,0.00,0.00,0.00,8.00,-15.70,0.00,0.10,0.00,0.00,0.0,0.00
3,3.006740e+03,2.508490e+03,tradewars_2,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
4,2.640008e+03,2.610008e+03,tradewars_0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
5,9.969089e+06,9.849758e+06,tradewars_mark,0,55.32,-107.34,81.11,-16.44,-92.31,34.80,64.81,-17.28,24.41,0.00,11.28,-0.7,33.70
6,1.002879e+07,9.953846e+06,tradewars_bob,0,-78.10,84.78,-103.97,17.14,93.01,-40.33,-150.61,-50.14,-58.21,0.68,0.00,0.0,0.00
7,9.385069e+02,9.385069e+02,etf_arbitrage,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
8,3.899843e+03,2.822088e+03,alphaforge_arbbot,0,0.02,0.00,0.00,-1.50,-1.50,-5.67,0.67,0.20,0.09,0.00,0.00,1.5,-0.09
9,1.937434e+03,2.720106e+02,bonds,0,0.00,0.00,0.30,0.00,0.00,0.00,0.00,0.00,0.00,-0.50,0.00,0.0,0.00


In [3]:
def get_market_positions(market_name: str):
    df = get_market_trades(market_name)
    if df.empty:
        return pd.DataFrame()
    return pd.concat([
        df.groupby("buyer")["size"].sum(),
        -df.groupby("seller")["size"].sum()
    ]).groupby(level=0).sum()

def get_multiple_market_positions(market_names: list[str]):
    return pd.DataFrame({market: get_market_positions(market) for market in market_names}).fillna(0)

In [ ]:
import ipywidgets as widgets

def get_dice(refresh):
    markets = ["min_dice_c", "max_dice_c", "avg_dice_c"]
    return get_multiple_market_positions(markets)

widgets.interact(get_dice, refresh=widgets.Checkbox(description="Refresh Data"))